In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import speech_recognition as sr
import tkinter

In [6]:
def get_roster(team):
    url = f"https://data.nba.com/data/v2022/json/mobile_teams/nba/2022/teams/{team}_roster.json"
    res = requests.get(url)
    j = res.json()
    players = j['data']['t']['pl']
    return [{
        "name": f"{player['fn']} {player['ln']}",
        "number": player['num'],
        "pos": player['pos'],
    } for player in players]

In [7]:
position_order = {
    "PG": 1,
    "SG": 2,
    "SF": 3,
    "PF": 4,
    "C": 5,
}

In [8]:
team_endpoints = {
    'celtics': 'bos/boston-celtics',
    'hawks': 'atl/atlanta-hawks',
    'bucks': 'mil/milwaukee-bucks',
    'bulls': 'chi/chicago-bulls',
    'cavaliers': 'cle/cleveland-cavaliers',
    'clippers': 'lac/la-clippers',
    'grizzlies': 'mem/memphis-grizzlies',
    'heat': 'mia/miami-heat',
    'hornets': 'cha/charlotte-hornets',
    'jazz': 'uta/utah-jazz',
    'kings': 'sac/sacramento-kings',
    'knicks': 'nyk/new-york-knicks',
    'lakers': 'lal/los-angeles-lakers',
    'magic': 'orl/orlando-magic',
    'mavericks': 'dal/dallas-mavericks',
    'nets': 'bkn/brooklyn-nets',
    'nuggets': 'den/denver-nuggets',
    'pacers': 'ind/indiana-pacers',
    'pelicans': 'nor/new-orleans-pelicans',
    'pistons': 'det/detroit-pistons',
    'raptors': 'tor/toronto-raptors',
    'rockets': 'hou/houston-rockets',
    'sixers': 'phi/philadelphia-76ers',
    'spurs': 'sas/san-antonio-spurs',
    'suns': 'phx/phoenix-suns',
    'thunder': 'okc/oklahoma-city-thunder',
    'timberwolves': 'min/minnesota-timberwolves',
    'warriors': 'gsw/golden-state-warriors',
    'wizards': 'was/washington-wizards',
}

In [9]:
def get_depth_chart(team_endpoint):
    url = f"https://www.espn.com/nba/team/depth/_/name/{team_endpoint}"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find_all("div", {"class": "nfl-depth-table"})[0]
    depth_chart = {}
    for i in range(5):
        position, row = table.find_all("tr", attrs={"data-idx": f"{i}"})
        players = row.findAll("a", attrs={"class": "AnchorLink"})
        depth_chart[position.text.strip()] = [player.text for player in players]
    return depth_chart

In [10]:
def get_players_depth_ranks(depth_chart):
    player_depth_ranks = {}
    for position, players in depth_chart.items():
        for i, player in enumerate(players):
            pos_rank = position_order[position]
            if player not in player_depth_ranks:
                cur_rank = float('inf'), pos_rank
            else:
                cur_rank = player_depth_ranks[player]
            new_rank = i, pos_rank
            player_depth_ranks[player] = min(new_rank, cur_rank)
    return player_depth_ranks

In [11]:
roster_data = get_roster('celtics')

In [12]:
depth_chart = get_depth_chart(team_endpoints['celtics'])
player_ranks = get_players_depth_ranks(depth_chart)

In [13]:
roster_data.sort(key=lambda player: player_ranks.get(player['name'], (float('inf'), float('inf'))))

In [14]:
roster_df = pd.DataFrame(roster_data)

In [15]:
def get_roster_dataframe(team):
    team_abbr = team_roster_hrefs[team]
    roster_data = get_roster(team_abbr)
    depth_chart = get_depth_chart(team_endpoints[team_abbr])
    player_ranks = get_players_depth_ranks(depth_chart)
    roster_data.sort(key=lambda player: player_ranks.get(player['name'], (float('inf'), float('inf'))))
    return pd.DataFrame(roster_data)

In [16]:
for team in team_roster_hrefs:
    try:
        get_roster_dataframe(team)
    except:
        print(team)

NameError: name 'team_roster_hrefs' is not defined

In [17]:
def swap_players(dataframe, index1, index2):
    tmp = dataframe.iloc[index1].copy()
    dataframe.iloc[index1] = dataframe.iloc[index2]
    dataframe.iloc[index2] = tmp

In [18]:
stats_df = pd.read_csv("data/nba-reference-per-game-2023.csv", index_col=0)

In [19]:
per_game_stats = {}
for i, data in stats_df.iterrows():
    player, rest = data['Player'], data[1:]
    per_game_stats[player] = rest.to_dict()

In [20]:
for player in player_ranks:
    if player not in per_game_stats:
        print(player)

Robert Williams III


In [21]:
special_names = {
    "Robert Williams": "Robert Williams III",
}

In [22]:
stats_df[stats_df['Player'] == 'Marcus Smart']

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player-additional
Rk,,,,,,,,,,,,,,,,,,,,,
449,Marcus Smart,PG,28,BOS,61,61,32.1,4.1,9.9,0.415,...,0.8,2.4,3.1,6.3,1.5,0.4,2.3,2.8,11.5,smartma01


In [23]:
name = 'Marcus Smart'
pd.DataFrame.from_dict({name: per_game_stats[name]}, orient='index')

,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player-additional
Marcus Smart,PG,28,BOS,61,61,32.1,4.1,9.9,0.415,1.9,...,0.8,2.4,3.1,6.3,1.5,0.4,2.3,2.8,11.5,smartma01


In [24]:
r = sr.Recognizer()
with sr.Microphone() as source:
    # read the audio data from the default microphone
    audio_data = r.record(source, duration=5)
    print("Recognizing...")
    # convert speech to text
    text = r.recognize_google(audio_data)
    print(text)

Recognizing...


UnknownValueError: 

In [26]:
def record_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        # read the audio data from the default microphone
        audio_data = r.record(source, duration=5)
        print("Recognizing...")
        # convert speech to text
        text = r.recognize_google(audio_data)
        print(text)
        return text
    

In [ ]:
record_audio()

Recognizing...
show me the Minnesota Timberwolves


'show me the Minnesota Timberwolves'

In [27]:
def get_team_roster_hrefs():
    url = f"https://www.nba.com/teams"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    teams = soup.find_all("a", {"class": "Anchor_anchor__cSc3P TeamFigure_tfMainLink__OPLFu"})

    team_roster_hrefs = {}
    for team in teams:
        href = team['href'].split('/')[1]
        team_roster_hrefs[team.text] = href

    return team_roster_hrefs

In [28]:
team_roster_hrefs = get_team_roster_hrefs()

In [29]:
team_roster_hrefs

{'Boston Celtics': 'celtics',
 'Brooklyn Nets': 'nets',
 'New York Knicks': 'knicks',
 'Philadelphia 76ers': 'sixers',
 'Toronto Raptors': 'raptors',
 'Chicago Bulls': 'bulls',
 'Cleveland Cavaliers': 'cavaliers',
 'Detroit Pistons': 'pistons',
 'Indiana Pacers': 'pacers',
 'Milwaukee Bucks': 'bucks',
 'Atlanta Hawks': 'hawks',
 'Charlotte Hornets': 'hornets',
 'Miami Heat': 'heat',
 'Orlando Magic': 'magic',
 'Washington Wizards': 'wizards',
 'Denver Nuggets': 'nuggets',
 'Minnesota Timberwolves': 'timberwolves',
 'Oklahoma City Thunder': 'thunder',
 'Portland Trail Blazers': 'blazers',
 'Utah Jazz': 'jazz',
 'Golden State Warriors': 'warriors',
 'LA Clippers': 'clippers',
 'Los Angeles Lakers': 'lakers',
 'Phoenix Suns': 'suns',
 'Sacramento Kings': 'kings',
 'Dallas Mavericks': 'mavericks',
 'Houston Rockets': 'rockets',
 'Memphis Grizzlies': 'grizzlies',
 'New Orleans Pelicans': 'pelicans',
 'San Antonio Spurs': 'spurs'}

In [30]:
def example_interaction():
    state = 0
    while True:
        match state:
            case 0:
                print("What team roster would you like to see?")
                team = record_audio()
                roster_df = get_roster_dataframe(team)
                print(roster_df)
                state = 1
            case 1:
                break

In [ ]:
example_interaction()

What team roster would you like to see?
Recognizing...
Atlanta Hawks
                 name number  pos
0          Trae Young     11    G
1     Dejounte Murray      5    G
2     De'Andre Hunter     12  F-G
3        John Collins     20  F-C
4        Clint Capela     15    C
5   Bogdan Bogdanovic     13    G
6          Saddiq Bey     41    F
7       Jalen Johnson      1    F
8      Onyeka Okongwu     17  F-C
9       Aaron Holiday      3    G
10         AJ Griffin     14    F
11     Bruno Fernando      0  F-C
12         Vit Krejci     27    G
13   Garrison Mathews     25    G
14      Trent Forrest      2    G
15      Tyrese Martin     22    G
16   Donovan Williams           G


In [31]:
def save_dataframe(df):
    with open('display.html', mode="a") as f:
        f.write(df.to_html())

In [32]:
df = get_roster_dataframe('Boston Celtics')
display_dataframes(df, df.head())

In [41]:
def display_dataframes(df1, df2):
    with open('display.html', mode="w") as f:
        df1['versus'] = '     '
        joined_df = pd.concat([d.reset_index(drop=True) for d in [df1, df2]], axis=1)
        f.write("<div> Rosters: </div>")
        f.write(joined_df.to_html())
        f.write('<br/>')
        f.write("no stats")
        # f.write(df1.to_html())

In [42]:
display_dataframes(df, df.head())


In [45]:
data = {
            "name": "Marcus Smart",
            "shots_attempted": 0,
            "shots_made": 0,
            "field_goal_percentage": 0,
        }

pd.DataFrame.from_dict(data, orient='index').T

,name,shots_attempted,shots_made,field_goal_percentage
0,Marcus Smart,0,0,0


In [46]:
from google.cloud import bigquery

In [47]:
bqc = bigquery.Client(project='spurs-sp2018')

In [48]:
sql = """--sql
SELECT * FROM ss.shots LIMIT 1
"""

In [51]:
sql_player_id = """--sql
SELECT * FROM ss.players 
WHERE firstName = 'Jayson' 
LIMIT 1
"""

In [52]:
data = bqc.query(sql_player_id).to_dataframe()

In [55]:
tatum_id = "986b713a-b20b-4eb0-919e-c859d0508af7"

In [78]:
sql_shots = f"""
SELECT x + 50 as x, y + 25 as y, outcome FROM ss.shots 
WHERE shooter_id = '{tatum_id}'
LIMIT 100
"""

In [79]:
data = bqc.query(sql_shots).to_dataframe()

In [133]:
x1 = -100
y1 = -100
x2 = 100
y2 = 100
bqc = bigquery.Client(project="spurs-sp2018")
tatum_id = "986b713a-b20b-4eb0-919e-c859d0508af7"
sql = f"""
    SELECT x + 50 as x, y + 25 as y, outcome, fouled, shot_type FROM ss.shots 
    WHERE shooter_id = '{tatum_id}'
    and x between {x1} and {x2}
    and y between {y1} and {y2}
    and season = '2018'
"""
data = bqc.query(sql).to_dataframe()

In [136]:
data.shape

(1229, 5)

In [135]:
data

,x,y,outcome,fouled,shot_type
0,9.59,24.29,True,False,layup
1,10.28,6.36,False,False,jumper
2,26.85,22.68,False,False,jumper
3,9.18,24.72,True,False,layup
4,9.64,22.95,True,False,layup
...,...,...,...,...,...
1224,8.39,47.01,False,False,jumper
1225,19.50,47.50,True,False,jumper
1226,13.03,20.94,False,True,layup
1227,9.07,20.48,True,False,layup
